In [ ]:
from anemoi.datasets import open_dataset

In [ ]:
def plot_grid(ds, path, s=0.1, c="r", grids=None, point=None):
    import matplotlib.pyplot as plt
    import cartopy.crs as ccrs
    import numpy as np

    lats, lons = ds.latitudes, ds.longitudes

    fig = plt.figure(figsize=(9, 9))
    proj = ccrs.NearsidePerspective(
        central_latitude=50.0, central_longitude=-25.0, satellite_height=4e6
    )

    ax = plt.axes(projection=proj)

    def fill():
        # Make sure we have a full globe
        lons, lats = np.meshgrid(np.arange(-180, 180, 1), np.arange(-90, 90, 1))
        x, y, _ = proj.transform_points(
            ccrs.PlateCarree(), lons.flatten(), lats.flatten()
        ).T

        mask = np.invert(np.logical_or(np.isinf(x), np.isinf(y)))
        x = np.compress(mask, x)
        y = np.compress(mask, y)

        # ax.tricontourf(x, y, values)
        ax.scatter(x, y, s=0, c="w")

    fill()

    def plot(what, s, c):
        x, y, _ = proj.transform_points(ccrs.PlateCarree(), lons[what], lats[what]).T

        mask = np.invert(np.logical_or(np.isinf(x), np.isinf(y)))
        x = np.compress(mask, x)
        y = np.compress(mask, y)

        # ax.tricontourf(x, y, values)
        ax.scatter(x, y, s=s, c=c)

    if grids:
        a = 0
        for i, b in enumerate(grids):
            if s[i] is not None:
                plot(slice(a, b), s[i], c[i])
            a += b
    else:
        plot(..., s, c)

    if point:
        point = np.array(point, dtype=np.float64)
        x, y, _ = proj.transform_points(ccrs.PlateCarree(), point[1], point[0]).T
        ax.scatter(x, y, s=100, c="k")

    ax.coastlines()

    if isinstance(path, str):
        fig.savefig(path, bbox_inches="tight")
    else:
        for p in path:
            fig.savefig(p, bbox_inches="tight")

In [ ]:
ds = open_dataset("grids1.zarr")

In [ ]:
plot_grid(ds, ["thinning-before.png", "cutout-1.png"], s=0.5)

In [ ]:
ds = open_dataset(ds, thinning=4)

In [ ]:
plot_grid(ds, "thinning-after.png", s=1, c="b")

In [ ]:
ds = open_dataset(cutout=["grids2.zarr", "grids1.zarr"])


plot_grid(
    ds,
    "cutout-4.png",
    s=[0.5, 0.5],
    grids=ds.grids,
    c=["g", "r"],
)

In [ ]:
plot_grid(ds, "cutout-2.png", s=[0.5, None], grids=ds.grids, c=["g", "r"])

In [ ]:
plot_grid(ds, "cutout-3.png", s=[None, 0.5], grids=ds.grids, c=["g", "r"])

In [ ]:
ds = open_dataset("grids1.zarr")
ds = open_dataset(ds, area=(60, -50, 20, 0))
plot_grid(ds, "area-1.png", s=0.5)